<a href="https://colab.research.google.com/github/NerminHussein/HR-Analytics_Classifier/blob/main/HR_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HR Analytics**








# **Problem Statement**

A random large company, employs at any given point of time, around 4000 employees. However, every year, around 15% of its employees leave the company and need to be replaced with the talent pool available in the job market. The management believes that this level of attrition (employees leaving, either on their own or because they got fired) is bad for the company, because of the following reasons -

The former employees’ projects get delayed, which makes it difficult to meet timelines, resulting in a reputation loss among consumers and partners
A sizeable department has to be maintained, for the purposes of recruiting new talent
More often than not, the new employees have to be trained for the job and/or given time to acclimatise themselves to the company
Hence, the management has contracted an HR analytics firm to understand what factors they should focus on, in order to curb attrition. In other words, they want to know what changes they should make to their workplace, in order to get most of their employees to stay. Also, they want to know which of these variables is most important and needs to be addressed right away.

Since you are one of the star analysts at the firm, this project has been given to you.

Goal of the case study
You are required to model the probability of attrition using a logistic regression. The results thus obtained will be used by the management to understand what changes they should make to their workplace, in order to get most of their employees to stay.




In [ ]:
# Getting the model ready by importing the libraries that will be used
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime,timedelta

In [ ]:
employee_survey_data=pd.read_csv('employee_survey_data.csv')
print(employee_survey_data.isnull().sum())


EmployeeID                  0
EnvironmentSatisfaction    25
JobSatisfaction            20
WorkLifeBalance            38
dtype: int64


In [ ]:
# Summary of Data
print(employee_survey_data.describe())

        EmployeeID  EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance
count  4410.000000              4385.000000      4390.000000      4372.000000
mean   2205.500000                 2.723603         2.728246         2.761436
std    1273.201673                 1.092756         1.101253         0.706245
min       1.000000                 1.000000         1.000000         1.000000
25%    1103.250000                 2.000000         2.000000         2.000000
50%    2205.500000                 3.000000         3.000000         3.000000
75%    3307.750000                 4.000000         4.000000         3.000000
max    4410.000000                 4.000000         4.000000         4.000000


In [ ]:
print(employee_survey_data.mean())
#mean                  2.723603         2.728246         2.761436
print(employee_survey_data.median())
#EnvironmentSatisfaction       3.0
#JobSatisfaction               3.0
#WorkLifeBalance               3.0

# Mean & Median Approximatery same so we choose Median to Fill the NAN Data

EmployeeID                 2205.500000
EnvironmentSatisfaction       2.723603
JobSatisfaction               2.728246
WorkLifeBalance               2.761436
dtype: float64
EmployeeID                 2205.5
EnvironmentSatisfaction       3.0
JobSatisfaction               3.0
WorkLifeBalance               3.0
dtype: float64


In [ ]:
#Fill Null in column of EnvironmentSatisfaction with Median
employee_survey_data['EnvironmentSatisfaction'] = employee_survey_data['EnvironmentSatisfaction'].fillna(employee_survey_data['EnvironmentSatisfaction'].median())

In [ ]:
#Fill Null in column of JobSatisfaction with Median
employee_survey_data['JobSatisfaction'] = employee_survey_data['JobSatisfaction'].fillna(employee_survey_data['JobSatisfaction'].median())

In [ ]:
#Fill Null in column of WorkLifeBalance with Median
employee_survey_data['WorkLifeBalance'] = employee_survey_data['WorkLifeBalance'].fillna(employee_survey_data['WorkLifeBalance'].median())

In [ ]:
#check if found NULL Values in employee_survey_data file 
print(employee_survey_data.isnull().sum())

EmployeeID                 0
EnvironmentSatisfaction    0
JobSatisfaction            0
WorkLifeBalance            0
dtype: int64


In [ ]:
Manager_survey_data=pd.read_csv('manager_survey_data.csv')
print(Manager_survey_data.isnull().sum())

EmployeeID           0
JobInvolvement       0
PerformanceRating    0
dtype: int64


In [ ]:
General_Data=pd.read_csv('general_data.csv')
print(General_Data.isnull().sum())

Age                         0
Attrition                   0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeID                  0
Gender                      0
JobLevel                    0
JobRole                     0
MaritalStatus               0
MonthlyIncome               0
NumCompaniesWorked         19
Over18                      0
PercentSalaryHike           0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
dtype: int64


In [ ]:
print(General_Data.mean())
#################################################
print("##########################")
print(General_Data.median())

Age                           36.923810
DistanceFromHome               9.192517
Education                      2.912925
EmployeeCount                  1.000000
EmployeeID                  2205.500000
JobLevel                       2.063946
MonthlyIncome              65029.312925
NumCompaniesWorked             2.694830
PercentSalaryHike             15.209524
StandardHours                  8.000000
StockOptionLevel               0.793878
TotalWorkingYears             11.279936
TrainingTimesLastYear          2.799320
YearsAtCompany                 7.008163
YearsSinceLastPromotion        2.187755
YearsWithCurrManager           4.123129
dtype: float64
##########################
Age                           36.0
DistanceFromHome               7.0
Education                      3.0
EmployeeCount                  1.0
EmployeeID                  2205.5
JobLevel                       2.0
MonthlyIncome              49190.0
NumCompaniesWorked             2.0
PercentSalaryHike             14.0
Sta

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  after removing the cwd from sys.path.


In [ ]:
General_Data['NumCompaniesWorked'] = General_Data['NumCompaniesWorked'].fillna(General_Data['NumCompaniesWorked'].mean())

In [ ]:
General_Data['TotalWorkingYears'] = General_Data['TotalWorkingYears'].fillna(General_Data['TotalWorkingYears'].mean())

In [ ]:
print(General_Data.isnull().sum())

Age                        0
Attrition                  0
BusinessTravel             0
Department                 0
DistanceFromHome           0
Education                  0
EducationField             0
EmployeeCount              0
EmployeeID                 0
Gender                     0
JobLevel                   0
JobRole                    0
MaritalStatus              0
MonthlyIncome              0
NumCompaniesWorked         0
Over18                     0
PercentSalaryHike          0
StandardHours              0
StockOptionLevel           0
TotalWorkingYears          0
TrainingTimesLastYear      0
YearsAtCompany             0
YearsSinceLastPromotion    0
YearsWithCurrManager       0
dtype: int64


In [ ]:
Merge_Employee_Manager=employee_survey_data.merge(Manager_survey_data, on='EmployeeID')

In [ ]:
Merge_Employee_Manager

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,1,3.0,4.0,2.0,3,3
1,2,3.0,2.0,4.0,2,4
2,3,2.0,2.0,1.0,3,3
3,4,4.0,4.0,3.0,2,3
4,5,4.0,1.0,3.0,3,3
...,...,...,...,...,...,...
4405,4406,4.0,1.0,3.0,3,3
4406,4407,4.0,4.0,3.0,2,3
4407,4408,1.0,3.0,3.0,3,4
4408,4409,4.0,1.0,3.0,2,3


In [ ]:
print(Merge_Employee_Manager.isnull().sum())

EmployeeID                 0
EnvironmentSatisfaction    0
JobSatisfaction            0
WorkLifeBalance            0
JobInvolvement             0
PerformanceRating          0
dtype: int64


In [ ]:
Merge_EM_Data=Merge_Employee_Manager.merge(General_Data, on='EmployeeID')

In [ ]:
Merge_EM_Data

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Age,Attrition,BusinessTravel,Department,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,1,3.0,4.0,2.0,3,3,51,No,Travel_Rarely,Sales,...,1.0,Y,11,8,0,1.000000,6,1,0,0
1,2,3.0,2.0,4.0,2,4,31,Yes,Travel_Frequently,Research & Development,...,0.0,Y,23,8,1,6.000000,3,5,1,4
2,3,2.0,2.0,1.0,3,3,32,No,Travel_Frequently,Research & Development,...,1.0,Y,15,8,3,5.000000,2,5,0,3
3,4,4.0,4.0,3.0,2,3,38,No,Non-Travel,Research & Development,...,3.0,Y,11,8,3,13.000000,5,8,7,5
4,5,4.0,1.0,3.0,3,3,32,No,Travel_Rarely,Research & Development,...,4.0,Y,12,8,2,9.000000,2,6,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,4406,4.0,1.0,3.0,3,3,42,No,Travel_Rarely,Research & Development,...,3.0,Y,17,8,1,10.000000,5,3,0,2
4406,4407,4.0,4.0,3.0,2,3,29,No,Travel_Rarely,Research & Development,...,2.0,Y,15,8,0,10.000000,2,3,0,2
4407,4408,1.0,3.0,3.0,3,4,25,No,Travel_Rarely,Research & Development,...,0.0,Y,20,8,0,5.000000,4,4,1,2
4408,4409,4.0,1.0,3.0,2,3,42,No,Travel_Rarely,Sales,...,0.0,Y,14,8,1,10.000000,2,9,7,8


In [ ]:
print(Merge_EM_Data.isnull().sum())

**Time analysis**

In [ ]:
log_in=pd.read_csv('in_time.csv')
log_out=pd.read_csv('out_time.csv')
# Finding the columns where it contains only null values which indicates it being a day off for the company 
#Doing that for the log in Dataframe
for columnName,columnValue in log_in.iteritems():
  nu=columnValue.isnull().sum()
  if nu==len(log_in):
    log_in.drop([columnName],axis=1,inplace=True)
#Doing the same for logout Dataframe
for columnName,columnValue in log_out.iteritems():
  nu=columnValue.isnull().sum()
  if nu==len(log_out):
    log_out.drop([columnName],axis=1,inplace=True)
#Making sure the remaining null values are equal in both Dataframes
#This could indicate day off for the employees
nu_in=sum(log_in.isnull().sum())
nu_out=sum(log_out.isnull().sum())
print(a==b)

True


In [ ]:
#Filling the null values with 0 to compre them later
log_in.replace(np.nan, 0)
log_out.replace(np.nan, 0)
#Comparing values from same column and index and if they match it will be replaced with null values
comp_df=log_in.compare(log_out,keep_shape=True)
#
nu_comp=(sum(comp_df.isnull().sum())/2)-len(log_in)
print(nu_comp==nu_in)


True


In [ ]:
work_hours_dict={}
for (idx,s1),(_,s2) in zip(log_in.iterrows(),log_out.iterrows()):
  lis=[]
  for (id,v1),(_,v2) in zip(s1.iteritems(),s2.iteritems()):
    a=str(v1).split()
    b=str(v2).split()
    if len(a)==2:
      c=datetime.strptime(a[1],'%H:%M:%S')
      d=datetime.strptime(b[1],'%H:%M:%S')
      tmdf=d-c
      tmdh=timedelta.total_seconds(tmdf)/3600
      lis.append(float(tmdh))
    else:
      lis.append('Day off')
  work_hours_dict[idx]=lis
work_hourst=pd.DataFrame.from_dict(work_hours_dict)


,0,1,2,3,4,5,6,7,8,9,...,4400,4401,4402,4403,4404,4405,4406,4407,4408,4409
0,Day off,Day off,Day off,Day off,Day off,Day off,Day off,Day off,Day off,Day off,...,Day off,Day off,Day off,Day off,Day off,Day off,Day off,Day off,Day off,Day off
1,7.208333,8.109167,6.6925,7.338333,8.055556,10.779444,6.838333,7.090278,Day off,7.360556,...,7.514167,7.756389,6.984167,7.500278,6.841111,8.118056,6.255556,7.276111,9.525556,6.836111
2,7.189722,7.454722,7.265556,7.291944,7.988056,10.721944,7.538333,7.053611,6.983611,7.101389,...,7.635278,7.991111,7.235278,7.294722,7.602778,8.840833,Day off,7.585278,9.586944,7.28
3,7.410833,Day off,6.405278,6.943056,7.6825,10.963611,7.266667,6.6525,7.318056,7.515,...,7.580556,8.140556,7.475,7.168056,7.44,8.399444,5.393611,7.641389,9.788333,6.727778
4,7.006667,7.396944,6.765,6.919444,7.806111,10.298611,6.421944,6.826111,7.557222,7.153056,...,7.115278,8.318056,6.630833,7.076111,7.013611,8.845,5.727778,7.986111,9.391111,6.842222


In [ ]:
#The resulted dataframe is transposed so we transpose it back
work_hours=work_hourst.T
work_hours.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,Day off,7.208333,7.189722,7.410833,7.006667,7.289722,7.484444,7.262778,7.831111,7.346944,...,Day off,Day off,7.339167,7.395833,6.504722,7.596389,7.773889,7.315,7.778889,7.080278
1,Day off,8.109167,7.454722,Day off,7.396944,7.416667,7.150833,7.611389,7.278889,7.613056,...,7.928333,7.903056,7.753889,7.712222,7.435556,Day off,7.614722,7.9825,7.986111,8.227222
2,Day off,6.6925,7.265556,6.405278,6.765,7.345,6.861389,7.418611,6.999722,7.438333,...,7.218333,6.785833,7.163611,6.801667,6.730278,6.849722,7.023889,7.438889,7.538889,6.786389
3,Day off,7.338333,7.291944,6.943056,6.919444,6.850833,7.193056,6.998611,7.306389,6.876667,...,7.455833,7.629167,6.846667,7.326389,7.413611,7.085,7.447222,7.416667,7.366389,7.133056
4,Day off,8.055556,7.988056,7.6825,7.806111,7.662222,7.721667,8.365,8.257222,8.26,...,8.32,7.903611,7.665,7.9575,7.786944,8.249444,7.662222,8.268611,7.953333,8.018056
